### "Simplification" attemps with OSMnx

For test (continents) cities [FUA ID]:
* (Africa) Douala [809]
* (Oceania) Auckland [869]
* (Asia) Aleppo [1133] -- Wuhan [8989]
* (Europe) Liège [1656]
* (South America) Bucaramanga [4617]
* (North America) Salt Lake City [4881]

In [1]:
%load_ext watermark
%load_ext memory_profiler
%watermark

Last updated: 2025-03-25T12:57:53.747519+01:00

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.30.0

Compiler    : GCC 13.3.0
OS          : Linux
Release     : 6.8.0-52-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit



In [ ]:
import os
import time

import geopandas as gpd
import momepy
import networkx as nx
import osmnx as ox

from core import utils

%watermark -w
%watermark -iv

Watermark: 2.5.0

geopandas: 1.0.1
core     : 0.1.dev208+gcbb258b.d20250325
osmnx    : 2.0.0
momepy   : 0.9.1
networkx : 3.4.2



**Read in meta data**

In [6]:
# read in sample metadata
sample = utils.read_sample_data()
sample.head(2)

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry,continent,iso_a3
305,9129.0,1.0,8078,Gonda,1.0,IND,India,66.0,29.0,1.074100e+06,1.066419e+06,7680.678101,"POLYGON ((81.98398 27.19657, 81.99471 27.19657...",Asia,IND
91,7578.0,6.0,10577;10581;10583;10596;10605;10607,Chongqing,1.0,CHN,China,2267.0,618.0,6.036834e+06,5.157726e+06,879107.861057,"POLYGON ((106.23972 29.52328, 106.19622 29.523...",Asia,CHN


**Read in data for example city**

In [7]:
# loop through FUAs

for fua in utils.fua_city:
    city = utils.fua_city[fua]
    print(f"Simplifying {city}")

    # read in original data
    gdf = utils.read_original(fua)

    t1 = time.time()
    # convert to networkX graph
    G = momepy.gdf_to_nx(
        gdf_network=gdf,
        approach="primal",
        directed=True,
        integer_labels=True,
        length="length",
    )
    # print(utils.graph_s ize(f"Momepy-NetworkX Primal Graph: {city}", G))

    # simplify in OSMnx (ie removing interstitial nodes)
    G_simp = ox.simplify_graph(G)
    # print(
    #     utils.graph_size(f"Momepy-NetworkX Primal Graph (simplified): {city}", G_simp)
    # )

    # consolidate with default value 10m
    tol = 10
    G_cons = ox.consolidate_intersections(
        G=G_simp,
        tolerance=tol,
        rebuild_graph=True,
        dead_ends=True,
        reconnect_edges=True,
    )
    # print(utils.graph_size(f"Consolidated graph with tolerance {tol}m", G_cons))

    # convert into digraph (drop multiedges, choosing by lenght)
    G_cons_di = ox.convert.to_digraph(G_cons, weight="length")
    # print(utils.graph_size("Consolidated DiGraph", G_cons_di))

    # make gdf of edge geoms (can't use ox.gdf method on non-multi digraph)
    geoms = list(nx.get_edge_attributes(G_cons_di, "geometry").values())
    edges_cons = gpd.GeoDataFrame(geometry=geoms, crs=G_cons_di.graph["crs"])

    t2 = time.time()
    print(f"{city} simplified in {round(t2 - t1, 1)}s \n")

    # save result to subfolder
    os.makedirs(f"../../data/{fua}/osmnx/", exist_ok=True)
    edges_cons.to_parquet(f"../../data/{fua}/osmnx/{fua}.parquet")

    print(f"OSMnx simplification done for {city}")

Simplifying Aleppo
Aleppo simplified in 10.2s 

OSMnx simplification done for Aleppo
Simplifying Auckland
Auckland simplified in 6.2s 

OSMnx simplification done for Auckland
Simplifying Bucaramanga
Bucaramanga simplified in 9.0s 

OSMnx simplification done for Bucaramanga
Simplifying Douala
Douala simplified in 10.9s 

OSMnx simplification done for Douala
Simplifying Liège
Liège simplified in 8.4s 

OSMnx simplification done for Liège
Simplifying Salt Lake City
Salt Lake City simplified in 6.0s 

OSMnx simplification done for Salt Lake City
Simplifying Wuhan
Wuhan simplified in 9.6s 

OSMnx simplification done for Wuhan


In [ ]:
def _simplify(gdf):
    # convert to networkX graph
    g = momepy.gdf_to_nx(
        gdf_network=gdf,
        approach="primal",
        directed=True,
        integer_labels=True,
        length="length",
    )
    # print(utils.graph_s ize(f"Momepy-NetworkX Primal Graph: {city}", G))

    # simplify in OSMnx (ie removing interstitial nodes)
    g_simp = ox.simplify_graph(g)
    # print(
    #     utils.graph_size(f"Momepy-NetworkX Primal Graph (simplified): {city}", G_simp)
    # )

    # consolidate with default value 10m
    tol = 10
    g_cons = ox.consolidate_intersections(
        G=g_simp,
        tolerance=tol,
        rebuild_graph=True,
        dead_ends=True,
        reconnect_edges=True,
    )
    # print(utils.graph_size(f"Consolidated graph with tolerance {tol}m", G_cons))

    # convert into digraph (drop multiedges, choosing by lenght)
    g_cons_di = ox.convert.to_digraph(g_cons, weight="length")
    # print(utils.graph_size("Consolidated DiGraph", G_cons_di))

    # make gdf of edge geoms (can't use ox.gdf method on non-multi digraph)
    geoms = list(nx.get_edge_attributes(g_cons_di, "geometry").values())
    edges_cons = gpd.GeoDataFrame(geometry=geoms, crs=G_cons_di.graph["crs"])
    return edges_cons

In [9]:
# loop through FUAs

for fua in utils.fua_city:
    city = utils.fua_city[fua]
    print(f"Simplifying {city}")

    # read in original data
    gdf = utils.read_original(fua)

    %memit edges = _simplify(gdf)
    print(f"{city} simplified \n")

Simplifying Aleppo
peak memory: 1485.80 MiB, increment: 145.62 MiB
Aleppo simplified 

Simplifying Auckland
peak memory: 1398.23 MiB, increment: 32.03 MiB
Auckland simplified 

Simplifying Bucaramanga
peak memory: 1500.48 MiB, increment: 103.28 MiB
Bucaramanga simplified 

Simplifying Douala
peak memory: 1567.36 MiB, increment: 149.38 MiB
Douala simplified 

Simplifying Liège
peak memory: 1553.18 MiB, increment: 91.09 MiB
Liège simplified 

Simplifying Salt Lake City
peak memory: 1519.63 MiB, increment: 18.91 MiB
Salt Lake City simplified 

Simplifying Wuhan
peak memory: 1575.70 MiB, increment: 62.97 MiB
Wuhan simplified 

